In [108]:
import sys, os; sys.path.append('..')
import random
import pyzx as zx
import logging
from fractions import Fraction
Z = zx.VertexType.Z
X = zx.VertexType.X
SE = zx.EdgeType.SIMPLE
HE = zx.EdgeType.HADAMARD

In [109]:
def bialg(g, v, w):
    vnhd = [v1 for v1 in g.neighbors(v) if v1!=w]
    wnhd = [w1 for w1 in g.neighbors(w) if w1!=v]
    zx.rules.apply_rule(g, zx.rules.bialg, [(v,w,vnhd,wnhd)])
def pivot(g, v, w, bv=None, bw=None):
    bv=[bv] if bv else []
    bw=[bw] if bw else []
    zx.rules.apply_rule(g, zx.rules.pivot, [(v,w,bv,bw)])    
def add_id(g, v, w):
    q = (g.qubit(v) + g.qubit(w))/2
    r = (g.row(v) + g.row(w))/2
    et = SE if g.edge_type((v,w)) == HE else HE
    g.remove_edge((v,w))
    v1 = g.add_vertex(Z, qubit=q, row=r)
    g.add_edge((v,v1), edgetype=HE)
    g.add_edge((v1,w), edgetype=et)
    return v1
def lc(g, v):
    zx.rules.apply_rule(g, zx.rules.lcomp, [(v,list(g.neighbors(v)))])
def blc(g, v, b):
    v1 = add_id(g, v, b)
    g.set_row(v1, g.row(v))
    if g.phase(v) != Fraction(1,2) and g.phase(v) != Fraction(-1,2):
        v2 = add_id(g, v1, b)
        g.add_to_phase(v, Fraction(1,2))
        g.add_to_phase(v2, Fraction(-1,2))
    zx.rules.apply_rule(g, zx.rules.lcomp, [(v,list(g.neighbors(v)))])

def blc2(g, v, b):
    nhd = [w for w in g.neighbors(v) if w != b]
    for i in range(len(nhd)):
        g.add_to_phase(nhd[i], Fraction(-1,2))
        for j in range(i+1, len(nhd)):
            g.add_edge_smart((nhd[i],nhd[j]), HE)
    v1 = add_id(g, v, b)
    g.set_phase(v1, Fraction(1,2))
    
def unfuse(g, v):
    q = g.qubit(v)
    r = g.row(v)
    v1 = g.add_vertex(qubit=-1,row=r,ty=Z)
    v2 = g.add_vertex(qubit=-2,row=r,ty=Z)
    g.add_edge((v2,v1), edgetype=HE)
    g.add_edge((v1,v), edgetype=HE)
    g.set_phase(v2,g.phase(v))
    g.set_phase(v,0)

In [110]:
g = zx.Graph()

verts = {
    0: g.add_vertex(row=1,qubit=2),
    1: g.add_vertex(row=1,qubit=3),
    2: g.add_vertex(row=2,qubit=2,ty=Z),
    3: g.add_vertex(row=2,qubit=3,ty=Z),
    4: g.add_vertex(row=3,qubit=0,ty=Z,phase=1/4),
    5: g.add_vertex(row=3,qubit=1,ty=Z),
    8: g.add_vertex(row=6,qubit=2),
    9: g.add_vertex(row=6,qubit=3) }

edges = [(0,2), (1,3), (2,8), (3,9)] 
hedges = [(4,5), (2,5), (3,5), (2,3)]

for s,t in edges: g.add_edge((verts[s],verts[t]))
for s,t in hedges: g.add_edge((verts[s],verts[t]), edgetype=HE)
zx.draw(g, labels=True)

In [111]:
g1 = g.copy()
add_id(g1, 0, 2)
add_id(g1, 2, 6)
add_id(g1, 1, 3)
add_id(g1, 3, 7)
pivot(g1, 2,3)
g1.remove_edge((8,10))
g1.remove_edge((9,11))
zx.draw(g1, labels=True)
blc2(g1, 9, 6)
blc2(g1, 11, 7)
# add_id(g1, 0,8)
# pivot(g1, 5,8)
zx.draw(g1, labels=True)
#print(list(g1.neighbors(9)))

In [112]:
# APPLY BIALGEBRA RULE
g1 = g.copy()
bialg(g1, 4, 5)
zx.draw(g1)

In [104]:
g = zx.generate.cliffordT(5, 100)
zx.full_reduce(g)
g.normalize()
zx.draw(g)

In [25]:
pivot(g1, 5, 6, None, 8)

In [26]:
zx.draw(g1)

In [10]:
# APPLY PIVOT
pivot(g1, 4, 5)
zx.draw(g1, labels=True)

In [11]:
zx.to_rg(g1, select=lambda v: v in [6,7])
zx.draw(g1)

In [12]:
g1 = g.copy()
zx.to_gh(g1)
g1.set_phase(4, Fraction(1/4))
g1.set_phase(5, Fraction(3/4))
zx.draw(g1, labels=True)

In [13]:
zx.rules.match_pivot(g1)

[]

In [14]:
unfuse(g1, 4)
unfuse(g1, 5)
zx.draw(g1, labels=True)

In [15]:
pivot(g1, 4, 5)
zx.draw(g1)

In [16]:
g1 = g.copy()
zx.to_gh(g1)
v1 = g1.add_vertex(row=4,qubit=-1,phase=Fraction(3/4),ty=Z)
g1.add_edge((v1,5),edgetype=HE)
g1.set_phase(4, Fraction(1/4))
zx.draw(g1, labels=True)

In [17]:
unfuse(g1,4)
zx.draw(g1)

In [18]:
pivot(g1,4,5)
zx.draw(g1,labels=True)

In [19]:
unfuse(g1,10)
pivot(g1,10,11)
zx.draw(g1)